In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive')
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Mounted at /content/drive
Sat Jan 20 00:04:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------

In [ ]:
pip install tsfresh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 9.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import numpy as np
import sys
import tsfresh as tsf
from tsfresh.utilities.dataframe_functions import impute

In [ ]:
!pwd

/content/drive/MyDrive


In [ ]:
os.chdir("/content")

df_outf = pd.read_csv('combined_demand_df_1h.csv')
df_outf = df_outf.drop('time_slot', axis=1)

df_inf = pd.read_csv('combined_supply_df_1h.csv')
df_inf = df_inf.drop('time_slot', axis=1)

if(df_outf.shape[0] != df_inf.shape[0]): # check if dem and inf are same length
    print("outflow and inflow are NOT of same length")
    sys.exit()

In [ ]:
df_outf

,4,12,13,24,41,42,43,45,48,50,...,237,238,239,243,244,246,249,261,262,263
0,3.0,0.0,3.0,7.0,5.0,2.0,22.0,4.0,81.0,7.0,...,35.0,39.0,28.0,0.0,2.0,9.0,20.0,2.0,15.0,54.0
1,1.0,0.0,2.0,4.0,22.0,14.0,6.0,0.0,72.0,20.0,...,16.0,27.0,44.0,1.0,3.0,12.0,32.0,6.0,18.0,49.0
2,2.0,0.0,2.0,2.0,11.0,9.0,5.0,0.0,63.0,14.0,...,17.0,23.0,21.0,0.0,3.0,13.0,17.0,5.0,7.0,27.0
3,3.0,0.0,0.0,2.0,3.0,7.0,2.0,1.0,31.0,5.0,...,7.0,10.0,8.0,0.0,1.0,3.0,10.0,1.0,2.0,12.0
4,0.0,0.0,0.0,2.0,8.0,4.0,0.0,0.0,20.0,8.0,...,2.0,4.0,6.0,0.0,2.0,4.0,3.0,2.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24808,2.0,0.0,37.0,6.0,27.0,4.0,51.0,12.0,140.0,26.0,...,339.0,179.0,199.0,1.0,0.0,105.0,147.0,23.0,80.0,126.0
24809,5.0,0.0,37.0,10.0,12.0,1.0,88.0,7.0,128.0,19.0,...,394.0,119.0,191.0,1.0,1.0,93.0,159.0,26.0,67.0,112.0
24810,3.0,0.0,27.0,10.0,4.0,4.0,49.0,7.0,161.0,26.0,...,281.0,76.0,176.0,0.0,0.0,88.0,184.0,16.0,35.0,99.0
24811,9.0,0.0,14.0,9.0,12.0,4.0,37.0,11.0,175.0,24.0,...,183.0,36.0,94.0,0.0,0.0,53.0,190.0,12.0,21.0,60.0


In [ ]:
class create_raw_features(object):
    #def __init__(self,):

    def create_lag_features(ts,lags_list):
        lag_ts0 = ts.shift(periods = lags_list[0])
        lag_ts1 = ts.shift(periods = lags_list[1])
        return lag_ts0,lag_ts1

    def create_diff_ts_features(ts,diff_list):
        diff1_p1 = ts.diff(periods=diff_list[0])
        diff1_p2 = ts.diff(periods=diff_list[1])

        diff2_p1 = ts.diff(periods=diff_list[0]).diff()
        return diff1_p1,diff1_p2,diff2_p1

In [ ]:
"""
stack all the raw feature time series of clusters vertically

tsfresh features are then added to these raw features to enlarge the feature set
"""

dt = pd.Timestamp("2021-01-01") + pd.to_timedelta(range(df_inf.shape[0]), "H")
raw_feat_df = pd.DataFrame(columns=['time_idx','datetime','group','target','inflow_lag168','inflow_lag336','outflow_lag168','outflow_lag336','inflow','inflow_diff1_per1','inflow_diff1_per2','inflow_diff2_per1','outflow_lag1','outflow_lag2','outflow_lag3','outflow_lag4','outflow_lag24','outflow_lag48'])

lags_list = [168,336]
lags_list1 = [1,2]
lags_list2 = [3,4]
lags_list3 = [24,48]
diff_list = [1,2]

for c in df_inf.columns:
    temp_df = pd.DataFrame(columns=['time_idx','datetime','group','target','inflow_lag168','inflow_lag336','outflow_lag168','outflow_lag336','inflow','inflow_diff1_per1','inflow_diff1_per2','inflow_diff2_per1','outflow_lag1','outflow_lag2','outflow_lag3','outflow_lag4','outflow_lag24','outflow_lag48'])
    temp_df['time_idx'] = range(df_inf.shape[0])
    temp_df['datetime'] = dt
    temp_df['target'] = df_outf[c]
    temp_df['group'] = np.repeat(int(c),df_inf.shape[0])
    temp_df['inflow_lag168'],temp_df['inflow_lag336'] = create_raw_features.create_lag_features(df_inf[c],lags_list)
    temp_df['outflow_lag168'],temp_df['outflow_lag336'] = create_raw_features.create_lag_features(df_outf[c],lags_list)

    temp_df['inflow'] = df_inf[c]
    temp_df['inflow_diff1_per1'],temp_df['inflow_diff1_per2'], temp_df['inflow_diff2_per1'] = create_raw_features.create_diff_ts_features(df_inf[c],diff_list)
    temp_df['outflow_lag1'],temp_df['outflow_lag2'] = create_raw_features.create_lag_features(df_outf[c],lags_list1)
    temp_df['outflow_lag3'],temp_df['outflow_lag4'] = create_raw_features.create_lag_features(df_outf[c],lags_list2)
    temp_df['outflow_lag24'],temp_df['outflow_lag48'] = create_raw_features.create_lag_features(df_outf[c],lags_list3)

    raw_feat_df = pd.concat([raw_feat_df, temp_df],axis=0)


raw_feat_df = raw_feat_df.dropna().reset_index(drop=True)
raw_feat_df

,time_idx,datetime,group,target,inflow_lag168,inflow_lag336,outflow_lag168,outflow_lag336,inflow,inflow_diff1_per1,inflow_diff1_per2,inflow_diff2_per1,outflow_lag1,outflow_lag2,outflow_lag3,outflow_lag4,outflow_lag24,outflow_lag48
0,336,2021-01-15 00:00:00,4,0.0,1.0,8.0,1.0,3.0,1.0,-5.0,-6.0,-4.0,0.0,1.0,1.0,1.0,0.0,0.0
1,337,2021-01-15 01:00:00,4,0.0,1.0,13.0,2.0,1.0,0.0,-1.0,-6.0,4.0,0.0,0.0,1.0,1.0,2.0,0.0
2,338,2021-01-15 02:00:00,4,0.0,3.0,9.0,0.0,2.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
3,339,2021-01-15 03:00:00,4,0.0,0.0,5.0,0.0,3.0,0.0,-1.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,340,2021-01-15 04:00:00,4,1.0,1.0,2.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615477,24808,2023-10-31 16:00:00,263,126.0,206.0,226.0,136.0,120.0,153.0,-23.0,-17.0,-29.0,142.0,137.0,107.0,114.0,125.0,85.0
1615478,24809,2023-10-31 17:00:00,263,112.0,226.0,191.0,93.0,149.0,135.0,-18.0,-41.0,5.0,126.0,142.0,137.0,107.0,113.0,71.0
1615479,24810,2023-10-31 18:00:00,263,99.0,185.0,190.0,110.0,110.0,119.0,-16.0,-34.0,2.0,112.0,126.0,142.0,137.0,82.0,51.0
1615480,24811,2023-10-31 19:00:00,263,60.0,136.0,158.0,64.0,67.0,119.0,0.0,-16.0,16.0,99.0,112.0,126.0,142.0,77.0,43.0


In [ ]:
"""
Create TSFRESH Time series features

"""

win_lengths = [24] # hyperparameter

# drop all categorical columns

col_list = raw_feat_df.columns

feature_settings = {'fft_coefficient': [{'coeff': 0, 'attr': 'real'}, {'coeff': 1, 'attr': 'real'}],
                    #'mean': None,
                    #'standard_deviation': None,
                    'skewness': None,
                    'kurtosis': None,
                    #'minimum': None,
                    #'maximum': None,
                    #'sum_values': None,
                    #'variance': None,
                    #'absolute_sum_of_changes': None,
                    'count_above_mean': None,
                    'count_below_mean': None,
                    #'longest_strike_above_mean': None,
                    #'longest_strike_below_mean': None,
                    #'number_crossing_m': [{'m': 0}],
                    'autocorrelation': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
                    'partial_autocorrelation': [{'lag': 1}, {'lag': 2}, {'lag': 3}]}


for c in col_list[8:]: # select 1 numeric column at a time, leave out target and long lags
    print('time series name: ',c)
    for win in win_lengths:
        num_df = raw_feat_df.loc[:,['datetime','group',c]]
        print('num_df.shape', num_df.shape)
        df_rolled = tsf.utilities.dataframe_functions.roll_time_series(num_df,column_id='group',column_sort='datetime',max_timeshift=win)
        df_rolled.drop(columns=['group'], inplace=True)
        df_features = tsf.extract_features(df_rolled, column_id='id', column_sort='datetime',
                                            default_fc_parameters=feature_settings)
        del df_rolled # free-up the RAM

        # Remove any missing values
        imputed_features = impute(df_features)
        del df_features # free-up the RAM

        # Select a subset of relevant features using tsfresh's built-in feature selection function
        raw_feat_df.index= range(raw_feat_df.shape[0])
        imputed_features.index = range(raw_feat_df.shape[0])
        selected_features = tsf.select_features(imputed_features, raw_feat_df['target'])
        del imputed_features # free-up the RAM

        # Print the names of the selected features
        print('selected_features.columns',selected_features.columns)

        print('selected_features.columns: ',selected_features.columns)
        print('selected_features.shape: ',selected_features.shape)

        print('all_stns_df.columns: ',raw_feat_df.columns)
        print('all_stns_df.shape: ',raw_feat_df.shape)

        raw_feat_df = pd.concat([raw_feat_df,selected_features],axis=1)

        del num_df # free-up the RAM
        del selected_features # free-up the RAM

raw_feat_df.shape

time series name:  inflow
num_df.shape (1615482, 3)


Feature Extraction: 100%|██████████| 20/20 [13:09<00:00, 39.48s/it]


selected_features.columns Index(['inflow__fft_coefficient__attr_"real"__coeff_0',
       'inflow__fft_coefficient__attr_"real"__coeff_1', 'inflow__skewness',
       'inflow__kurtosis', 'inflow__count_above_mean',
       'inflow__count_below_mean', 'inflow__autocorrelation__lag_1',
       'inflow__autocorrelation__lag_2', 'inflow__autocorrelation__lag_3',
       'inflow__partial_autocorrelation__lag_1',
       'inflow__partial_autocorrelation__lag_2',
       'inflow__partial_autocorrelation__lag_3'],
      dtype='object')
selected_features.columns:  Index(['inflow__fft_coefficient__attr_"real"__coeff_0',
       'inflow__fft_coefficient__attr_"real"__coeff_1', 'inflow__skewness',
       'inflow__kurtosis', 'inflow__count_above_mean',
       'inflow__count_below_mean', 'inflow__autocorrelation__lag_1',
       'inflow__autocorrelation__lag_2', 'inflow__autocorrelation__lag_3',
       'inflow__partial_autocorrelation__lag_1',
       'inflow__partial_autocorrelation__lag_2',
       'inflow__

Feature Extraction: 100%|██████████| 20/20 [13:26<00:00, 40.33s/it]


selected_features.columns Index(['inflow_diff1_per1__fft_coefficient__attr_"real"__coeff_0',
       'inflow_diff1_per1__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff1_per1__kurtosis', 'inflow_diff1_per1__count_above_mean',
       'inflow_diff1_per1__count_below_mean',
       'inflow_diff1_per1__autocorrelation__lag_1',
       'inflow_diff1_per1__autocorrelation__lag_2',
       'inflow_diff1_per1__autocorrelation__lag_3',
       'inflow_diff1_per1__partial_autocorrelation__lag_1',
       'inflow_diff1_per1__partial_autocorrelation__lag_2',
       'inflow_diff1_per1__partial_autocorrelation__lag_3',
       'inflow_diff1_per1__skewness'],
      dtype='object')
selected_features.columns:  Index(['inflow_diff1_per1__fft_coefficient__attr_"real"__coeff_0',
       'inflow_diff1_per1__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff1_per1__kurtosis', 'inflow_diff1_per1__count_above_mean',
       'inflow_diff1_per1__count_below_mean',
       'inflow_diff1_per1__autocorr

...

Feature Extraction: 100%|██████████| 20/20 [13:08<00:00, 39.44s/it]


selected_features.columns Index(['outflow_lag24__fft_coefficient__attr_"real"__coeff_0',
       'outflow_lag24__fft_coefficient__attr_"real"__coeff_1',
       'outflow_lag24__skewness', 'outflow_lag24__kurtosis',
       'outflow_lag24__count_above_mean', 'outflow_lag24__count_below_mean',
       'outflow_lag24__autocorrelation__lag_1',
       'outflow_lag24__autocorrelation__lag_2',
       'outflow_lag24__autocorrelation__lag_3',
       'outflow_lag24__partial_autocorrelation__lag_1',
       'outflow_lag24__partial_autocorrelation__lag_2',
       'outflow_lag24__partial_autocorrelation__lag_3'],
      dtype='object')
selected_features.columns:  Index(['outflow_lag24__fft_coefficient__attr_"real"__coeff_0',
       'outflow_lag24__fft_coefficient__attr_"real"__coeff_1',
       'outflow_lag24__skewness', 'outflow_lag24__kurtosis',
       'outflow_lag24__count_above_mean', 'outflow_lag24__count_below_mean',
       'outflow_lag24__autocorrelation__lag_1',
       'outflow_lag24__autocorrelat

Feature Extraction: 100%|██████████| 20/20 [13:09<00:00, 39.48s/it]


selected_features.columns Index(['outflow_lag48__fft_coefficient__attr_"real"__coeff_0',
       'outflow_lag48__fft_coefficient__attr_"real"__coeff_1',
       'outflow_lag48__skewness', 'outflow_lag48__kurtosis',
       'outflow_lag48__count_above_mean', 'outflow_lag48__count_below_mean',
       'outflow_lag48__autocorrelation__lag_1',
       'outflow_lag48__autocorrelation__lag_2',
       'outflow_lag48__autocorrelation__lag_3',
       'outflow_lag48__partial_autocorrelation__lag_1',
       'outflow_lag48__partial_autocorrelation__lag_2',
       'outflow_lag48__partial_autocorrelation__lag_3'],
      dtype='object')
selected_features.columns:  Index(['outflow_lag48__fft_coefficient__attr_"real"__coeff_0',
       'outflow_lag48__fft_coefficient__attr_"real"__coeff_1',
       'outflow_lag48__skewness', 'outflow_lag48__kurtosis',
       'outflow_lag48__count_above_mean', 'outflow_lag48__count_below_mean',
       'outflow_lag48__autocorrelation__lag_1',
       'outflow_lag48__autocorrelat

(1615482, 137)

In [ ]:
for c in raw_feat_df.columns:
    print(c)

time_idx
datetime
group
target
inflow_lag168
inflow_lag336
outflow_lag168
outflow_lag336
inflow
inflow_diff1_per1
inflow_diff1_per2
inflow_diff2_per1
outflow_lag1
outflow_lag2
outflow_lag3
outflow_lag4
outflow_lag24
outflow_lag48
inflow__fft_coefficient__attr_"real"__coeff_0
inflow__fft_coefficient__attr_"real"__coeff_1
inflow__skewness
inflow__kurtosis
inflow__count_above_mean
inflow__count_below_mean
inflow__autocorrelation__lag_1
inflow__autocorrelation__lag_2
inflow__autocorrelation__lag_3
inflow__partial_autocorrelation__lag_1
inflow__partial_autocorrelation__lag_2
inflow__partial_autocorrelation__lag_3
inflow_diff1_per1__fft_coefficient__attr_"real"__coeff_0
inflow_diff1_per1__fft_coefficient__attr_"real"__coeff_1
inflow_diff1_per1__kurtosis
inflow_diff1_per1__count_above_mean
inflow_diff1_per1__count_below_mean
inflow_diff1_per1__autocorrelation__lag_1
inflow_diff1_per1__autocorrelation__lag_2
inflow_diff1_per1__autocorrelation__lag_3
inflow_diff1_per1__partial_autocorrelation__

In [ ]:
chunk_size = 100000  # or adjust based on available memory
num_chunks = len(raw_feat_df) // chunk_size + 1

for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(raw_feat_df))
    df_chunk = raw_feat_df.iloc[start_idx:end_idx, :]
    df_chunk.to_csv(f'chunk_{i + 1}.csv', index=False, header=True)

In [ ]:
raw_feat_df

,time_idx,datetime,group,target,inflow_lag168,inflow_lag336,outflow_lag168,outflow_lag336,inflow,inflow_diff1_per1,...,outflow_lag48__skewness,outflow_lag48__kurtosis,outflow_lag48__count_above_mean,outflow_lag48__count_below_mean,outflow_lag48__autocorrelation__lag_1,outflow_lag48__autocorrelation__lag_2,outflow_lag48__autocorrelation__lag_3,outflow_lag48__partial_autocorrelation__lag_1,outflow_lag48__partial_autocorrelation__lag_2,outflow_lag48__partial_autocorrelation__lag_3
0,336,2021-01-15 00:00:00,4,0.0,1.0,8.0,1.0,3.0,1.0,-5.0,...,0.108234,-0.981927,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
1,337,2021-01-15 01:00:00,4,0.0,1.0,13.0,2.0,1.0,0.0,-1.0,...,0.108234,-0.981927,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
2,338,2021-01-15 02:00:00,4,0.0,3.0,9.0,0.0,2.0,1.0,1.0,...,0.000000,-0.981927,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
3,339,2021-01-15 03:00:00,4,0.0,0.0,5.0,0.0,3.0,0.0,-1.0,...,0.000000,0.000000,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
4,340,2021-01-15 04:00:00,4,1.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615477,24808,2023-10-31 16:00:00,263,126.0,206.0,226.0,136.0,120.0,153.0,-23.0,...,-0.176387,-0.980240,14.0,11.0,0.849907,0.650842,0.354993,0.849907,-0.257512,-0.469614
1615478,24809,2023-10-31 17:00:00,263,112.0,226.0,191.0,93.0,149.0,135.0,-18.0,...,-0.033360,-0.933162,13.0,12.0,0.868410,0.608625,0.313220,0.868410,-0.591838,-0.088675
1615479,24810,2023-10-31 18:00:00,263,99.0,185.0,190.0,110.0,110.0,119.0,-16.0,...,0.129079,-0.974921,12.0,13.0,0.843547,0.578663,0.231796,0.843547,-0.460806,-0.407406
1615480,24811,2023-10-31 19:00:00,263,60.0,136.0,158.0,64.0,67.0,119.0,0.0,...,0.292567,-0.856106,12.0,13.0,0.836609,0.511540,0.157736,0.836609,-0.627740,-0.075322


In [ ]:
pip install pyarrow

In [ ]:
raw_feat_df.to_parquet('combined_dd_ss_raw_feat_data.parquet')

In [ ]:
pd.read_parquet('combined_dd_ss_raw_feat_data.parquet', engine='pyarrow')

,time_idx,datetime,group,target,inflow_lag168,inflow_lag336,outflow_lag168,outflow_lag336,inflow,inflow_diff1_per1,...,outflow_lag48__skewness,outflow_lag48__kurtosis,outflow_lag48__count_above_mean,outflow_lag48__count_below_mean,outflow_lag48__autocorrelation__lag_1,outflow_lag48__autocorrelation__lag_2,outflow_lag48__autocorrelation__lag_3,outflow_lag48__partial_autocorrelation__lag_1,outflow_lag48__partial_autocorrelation__lag_2,outflow_lag48__partial_autocorrelation__lag_3
0,336,2021-01-15 00:00:00,4,0.0,1.0,8.0,1.0,3.0,1.0,-5.0,...,0.108234,-0.981927,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
1,337,2021-01-15 01:00:00,4,0.0,1.0,13.0,2.0,1.0,0.0,-1.0,...,0.108234,-0.981927,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
2,338,2021-01-15 02:00:00,4,0.0,3.0,9.0,0.0,2.0,1.0,1.0,...,0.000000,-0.981927,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
3,339,2021-01-15 03:00:00,4,0.0,0.0,5.0,0.0,3.0,0.0,-1.0,...,0.000000,0.000000,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
4,340,2021-01-15 04:00:00,4,1.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615477,24808,2023-10-31 16:00:00,263,126.0,206.0,226.0,136.0,120.0,153.0,-23.0,...,-0.176387,-0.980240,14.0,11.0,0.849907,0.650842,0.354993,0.849907,-0.257512,-0.469614
1615478,24809,2023-10-31 17:00:00,263,112.0,226.0,191.0,93.0,149.0,135.0,-18.0,...,-0.033360,-0.933162,13.0,12.0,0.868410,0.608625,0.313220,0.868410,-0.591838,-0.088675
1615479,24810,2023-10-31 18:00:00,263,99.0,185.0,190.0,110.0,110.0,119.0,-16.0,...,0.129079,-0.974921,12.0,13.0,0.843547,0.578663,0.231796,0.843547,-0.460806,-0.407406
1615480,24811,2023-10-31 19:00:00,263,60.0,136.0,158.0,64.0,67.0,119.0,0.0,...,0.292567,-0.856106,12.0,13.0,0.836609,0.511540,0.157736,0.836609,-0.627740,-0.075322


In [ ]:
!pwd

/content


In [ ]:
os.chdir('/content/drive/MyDrive')

In [ ]:
!pwd

/content/drive/MyDrive


In [ ]:
raw_feat_df.to_parquet('combined_dd_ss_raw_feat_data.parquet')

In [ ]:
# os.chdir('/content/drive/MyDrive')
pd.read_parquet('combined_dd_ss_raw_feat_data.parquet', engine='pyarrow')

,time_idx,datetime,group,target,inflow_lag168,inflow_lag336,outflow_lag168,outflow_lag336,inflow,inflow_diff1_per1,...,outflow_lag48__skewness,outflow_lag48__kurtosis,outflow_lag48__count_above_mean,outflow_lag48__count_below_mean,outflow_lag48__autocorrelation__lag_1,outflow_lag48__autocorrelation__lag_2,outflow_lag48__autocorrelation__lag_3,outflow_lag48__partial_autocorrelation__lag_1,outflow_lag48__partial_autocorrelation__lag_2,outflow_lag48__partial_autocorrelation__lag_3
0,336,2021-01-15 00:00:00,4,0.0,1.0,8.0,1.0,3.0,1.0,-5.0,...,0.108234,-0.981927,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
1,337,2021-01-15 01:00:00,4,0.0,1.0,13.0,2.0,1.0,0.0,-1.0,...,0.108234,-0.981927,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
2,338,2021-01-15 02:00:00,4,0.0,3.0,9.0,0.0,2.0,1.0,1.0,...,0.000000,-0.981927,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
3,339,2021-01-15 03:00:00,4,0.0,0.0,5.0,0.0,3.0,0.0,-1.0,...,0.000000,0.000000,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
4,340,2021-01-15 04:00:00,4,1.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615477,24808,2023-10-31 16:00:00,263,126.0,206.0,226.0,136.0,120.0,153.0,-23.0,...,-0.176387,-0.980240,14.0,11.0,0.849907,0.650842,0.354993,0.849907,-0.257512,-0.469614
1615478,24809,2023-10-31 17:00:00,263,112.0,226.0,191.0,93.0,149.0,135.0,-18.0,...,-0.033360,-0.933162,13.0,12.0,0.868410,0.608625,0.313220,0.868410,-0.591838,-0.088675
1615479,24810,2023-10-31 18:00:00,263,99.0,185.0,190.0,110.0,110.0,119.0,-16.0,...,0.129079,-0.974921,12.0,13.0,0.843547,0.578663,0.231796,0.843547,-0.460806,-0.407406
1615480,24811,2023-10-31 19:00:00,263,60.0,136.0,158.0,64.0,67.0,119.0,0.0,...,0.292567,-0.856106,12.0,13.0,0.836609,0.511540,0.157736,0.836609,-0.627740,-0.075322


In [ ]:
"""
create train, validation and test sets
SAVE to csv.
"""


train_stop_time = '2021/11/30 00:00'
val_stop_time = '2021/12/16 00:00'


train = raw_feat_df.loc[raw_feat_df['datetime'] < train_stop_time]
val = raw_feat_df.loc[(raw_feat_df['datetime'] >= train_stop_time) & (raw_feat_df['datetime'] < val_stop_time)]
full_train = raw_feat_df.loc[raw_feat_df['datetime'] < val_stop_time]
test = raw_feat_df.loc[raw_feat_df['datetime'] >= val_stop_time]

"""# Linux
os.chdir('/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/2_freq_nbinom_LSTM/1_cluster_demand_prediction/data/demand_data/standalone/region_level/train_val_test_data')
"""

"""# Win"""
os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\\region_level\\train_val_test_split')
train.to_csv(region_name+'_all_clstr_train_data_<'+str(train_stop_time)+'.csv',index=False)
full_train.to_csv(region_name+'_all_clstr_full_train_data_<'+str(val_stop_time)+'.csv',index=False)
val.to_csv(region_name+'_all_clstr_val_data_<'+str(val_stop_time)+'.csv',index=False)
test.to_csv(region_name+'_all_clstr_test_data_>'+str(val_stop_time)+'.csv',index=False)
